# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments)

## 1. 데이터 전처리

In [1]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
df=pd.read_csv('data/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [3]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [4]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [5]:
headline = [title for title in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [6]:
len(headline)

1324

In [7]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

1214

In [8]:
# 구두점 제거와 동시에 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() 

In [9]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [10]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [11]:
sequences = []
for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [12]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : offer


In [14]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [15]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [16]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [17]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [19]:
X.shape, y.shape

((7803, 23), (7803, 3494))

## 2. 모델 설계 및 학습

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [21]:
embedding = Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer")
lstm = LSTM(128, name="LSTM_Layer")
output = Dense(vocab_size, activation='softmax', name="Output_Layer")

In [22]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
model = Sequential()
model.add(embedding)
model.add(lstm)
model.add(output)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(X, y, epochs=200, verbose=2)

Train on 7803 samples
Epoch 1/200
7803/7803 - 6s - loss: 7.6308 - accuracy: 0.0268
Epoch 2/200
7803/7803 - 4s - loss: 7.1229 - accuracy: 0.0270
Epoch 3/200
7803/7803 - 4s - loss: 6.9830 - accuracy: 0.0311
Epoch 4/200
7803/7803 - 4s - loss: 6.8664 - accuracy: 0.0411
Epoch 5/200
7803/7803 - 4s - loss: 6.7273 - accuracy: 0.0429
Epoch 6/200
7803/7803 - 4s - loss: 6.5741 - accuracy: 0.0478
Epoch 7/200
7803/7803 - 4s - loss: 6.4011 - accuracy: 0.0499
Epoch 8/200
7803/7803 - 4s - loss: 6.2129 - accuracy: 0.0572
Epoch 9/200
7803/7803 - 4s - loss: 6.0321 - accuracy: 0.0586
Epoch 10/200
7803/7803 - 4s - loss: 5.8548 - accuracy: 0.0634
Epoch 11/200
7803/7803 - 4s - loss: 5.6898 - accuracy: 0.0687
Epoch 12/200
7803/7803 - 4s - loss: 5.5271 - accuracy: 0.0741
Epoch 13/200
7803/7803 - 4s - loss: 5.3769 - accuracy: 0.0774
Epoch 14/200
7803/7803 - 4s - loss: 5.2247 - accuracy: 0.0838
Epoch 15/200
7803/7803 - 4s - loss: 5.0822 - accuracy: 0.0906
Epoch 16/200
7803/7803 - 4s - loss: 4.9433 - accuracy: 0.

Epoch 133/200
7803/7803 - 4s - loss: 0.3555 - accuracy: 0.9150
Epoch 134/200
7803/7803 - 4s - loss: 0.3527 - accuracy: 0.9138
Epoch 135/200
7803/7803 - 4s - loss: 0.3477 - accuracy: 0.9159
Epoch 136/200
7803/7803 - 4s - loss: 0.3480 - accuracy: 0.9145
Epoch 137/200
7803/7803 - 4s - loss: 0.3452 - accuracy: 0.9154
Epoch 138/200
7803/7803 - 4s - loss: 0.3495 - accuracy: 0.9143
Epoch 139/200
7803/7803 - 4s - loss: 0.3571 - accuracy: 0.9116
Epoch 140/200
7803/7803 - 4s - loss: 0.3550 - accuracy: 0.9117
Epoch 141/200
7803/7803 - 4s - loss: 0.3763 - accuracy: 0.9104
Epoch 142/200
7803/7803 - 4s - loss: 0.3438 - accuracy: 0.9138
Epoch 143/200
7803/7803 - 4s - loss: 0.3240 - accuracy: 0.9186
Epoch 144/200
7803/7803 - 4s - loss: 0.3179 - accuracy: 0.9182
Epoch 145/200
7803/7803 - 4s - loss: 0.3150 - accuracy: 0.9173
Epoch 146/200
7803/7803 - 4s - loss: 0.3128 - accuracy: 0.9180
Epoch 147/200
7803/7803 - 4s - loss: 0.3109 - accuracy: 0.9167
Epoch 148/200
7803/7803 - 4s - loss: 0.3114 - accuracy:

## 3. 모델 검증

In [25]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [26]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성

i cant jump ship from facebook yet shouts of rekindle us


In [27]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성

how do you get your nature fix be be ok to


In [28]:
print(sentence_generation(model, t, 'former', 10))

former fbi deputy director is faulted in scathing inspector general report


### Weights and bias 확인

In [29]:
embedding.get_config()

{'name': 'Embedding_Layer',
 'trainable': True,
 'batch_input_shape': (None, 23),
 'dtype': 'float32',
 'input_dim': 3494,
 'output_dim': 10,
 'embeddings_initializer': {'class_name': 'RandomUniform',
  'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}},
 'embeddings_regularizer': None,
 'activity_regularizer': None,
 'embeddings_constraint': None,
 'mask_zero': False,
 'input_length': 23}

In [30]:
lstm.get_config()

{'name': 'LSTM_Layer',
 'trainable': True,
 'dtype': 'float32',
 'return_sequences': False,
 'return_state': False,
 'go_backwards': False,
 'stateful': False,
 'unroll': False,
 'time_major': False,
 'units': 128,
 'activation': 'tanh',
 'recurrent_activation': 'sigmoid',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'unit_forget_bias': True,
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0,
 'implementation': 2}

In [32]:
lstmw = lstm.get_weights()
len(lstmw)

3

In [34]:
lstmw[0].shape, lstmw[1].shape, lstmw[2].shape

((10, 512), (128, 512), (512,))